In [1]:
from GoogleNews   import GoogleNews
from newspaper    import Article
from newspaper    import Config
import pandas     as pd
import nltk
import time
import random
import datetime
from datetime import datetime
from datetime import timedelta
from datetime import time
from datetime import date
import calendar

import pyodbc
import json
from pymongo import MongoClient

# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\maria.rocio.ucha\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Credenciales para conexión a Azure SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=tfmcice.database.windows.net;'
                      'Database=StocksDB;'
                      'UID=tfmcice;'
                      'PWD=Q1w2e3r4t5!;'
                      'Trusted_Connections=no;')

cursor = conn.cursor()
cursor.execute('''select s.stock, s.company_name
               from stocks s''') 

row = cursor.fetchone() 
stocks = []
while row: 
    # Columna 0: Ticket, Columna 1: Company Name
    stock_detail = { 'stock' : row[0], 'company_name': row[1]}
    stocks.append(stock_detail)
    row = cursor.fetchone()

cursor.execute('''select sources.source_name
                from news_sources sources''') 

row = cursor.fetchone() 
sources = []
while row: 
    sources.append(row[0])
    row = cursor.fetchone()

cursor.close()
    
print("Total de acciones a recuperar:", len(stocks))
print(stocks)
print("Fuentes Fiables")
print(sources)

Total de acciones a recuperar: 11
[{'stock': 'AXP', 'company_name': 'American Express Co'}, {'stock': 'BA', 'company_name': 'Boeing Company'}, {'stock': 'D', 'company_name': 'Dominion Energy'}, {'stock': 'DD', 'company_name': 'DuPont de Nemours Inc'}, {'stock': 'GOOG', 'company_name': 'Alphabet Inc.\xa0(Class C)'}, {'stock': 'IBM', 'company_name': 'International Business Machines'}, {'stock': 'KO', 'company_name': 'Coca-Cola Company'}, {'stock': 'PFE', 'company_name': 'Pfizer Inc.'}, {'stock': 'SPG', 'company_name': 'Simon Property Group Inc'}, {'stock': 'TSLA', 'company_name': 'Tesla, Inc.'}, {'stock': 'XOM', 'company_name': 'Exxon Mobil Corp.'}]
Fuentes Fiables
['Financial Times', 'The Wall Street Journal', 'International Business Times', 'Reuters', 'Business Insider', 'The economist', 'Bloomberg', 'Businessweek', 'Forbes', 'Fortune', 'CNN', 'Money CNN', 'Yahoo Finance', 'Euromoney', 'BBC', 'The New York Times', 'The Washington Post']


In [3]:
#config will allow us to access the specified url for which we are #not authorized. 
#Sometimes we may get 403 client error while parsing #the link to download the article.

nltk.download('punkt')

# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Connection Mongo DB
uri = "mongodb://tfmcice:i9PqjFS50Iad5WusUksnFnIjleXHo8vXjONtJuSHeDELDrC3nOjMIs6HWIADG7BE1aogkBEFGxzFWTZBgTEEyg==@tfmcice.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@tfmcice@"
client = MongoClient(uri)
db = client['StocksDB']
companies = db['Companies']

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 15
config.language = 'en'

for company in stocks:
    
    print("Company:",company.get("stock"),"-",company.get("company_name"))
   
    start = datetime(2018, 1, 1)
    end   = datetime(2021, 1, 1)

    month_list = [i for i in pd.date_range(start=start, end=end, freq='3MS')]
    for i in range(len(month_list)-1):
        print("***********************************************")
        startdate = month_list[i].strftime("%m/%d/%Y")
        print(startdate)
        todate = month_list[i+1]+timedelta(days = -1)
        enddate = todate.strftime("%m/%d/%Y")
        print(enddate)
        print("***********************************************")
    
        googlenews = GoogleNews(start=startdate,end=enddate)
        googlenews.set_lang('en')
        googlenews.search(company.get("company_name"))

        for i in range(1,20):       
            googlenews.getpage(i)
            result = googlenews.result()
            df = pd.DataFrame(result)

        if not(df.empty) :
            df = df[df.media.isin(sources)]   

        lista=[]
        for ind in df.index:

            dicc={'Company': company.get("company_name"),
                 'Ticker': company.get("stock")}

            article = Article(df['link'][ind], config= config)

            try:
                article.download()
                article.parse()
                article.nlp()

                dicc['Date']   =df['datetime'][ind].strftime("%b-%d-%Y %H:%M%p")
                dicc['Media']  =df['media'][ind]
                dicc['Title']  =article.title
                dicc['Article']=article.text
                dicc['Summary']=article.summary

                lista.append(dicc)

                time.sleep(random.randint(5,15))
            except Exception as e:
                print("No Pass URL: ", df['link'][ind], str(e))
                pass

        news_df = pd.DataFrame(lista)
        #news_df['Time'] = pd.to_datetime(news_df.Date).dt.time
        #news_df['Date'] = pd.to_datetime(news_df.Date).dt.date

        if not(news_df.empty) :
            # Iterate through the headlines and get the polarity scores using vader
            scores = news_df['Article'].apply(vader.polarity_scores).tolist()
            #Convert the 'scores' list of dicts into a DataFrame
            scores_df = pd.DataFrame(scores)

            # Join the DataFrames of the news and the list of dicts
            news_df = news_df.join(scores_df, rsuffix='_right')

            #path = "data/{}_PRU.xlsx".format(company)
            #news_df.to_excel(path,index=False)

            list_dict = []
            #with open('data/Companies_All.json', 'a+', encoding='utf-8') as fich:
            for index, row in list(news_df.iterrows()):
                list_dict.append(dict(row))

            companies.insert_many(list_dict)
            print("INSERT")
            #json.dump(list_dict, fich, ensure_ascii=False, indent=4) # , sort_keys=False

client.close()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maria.rocio.ucha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Company: AXP - American Express Co
***********************************************
01/01/2018
03/31/2018
***********************************************
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests


KeyboardInterrupt: 